#### Database structure:

Publications(pmid, year, title, abstract, ta_search) - 28M records, ta_search is a `tsvector` for full text search in both title and abstract

Citations(pmid_citing, pmid_cited) - approx. 60M records

CitationsCount(pmid, count) - can be created in 30+ minutes

In [2]:
import psycopg2 as pg_driver

In [3]:
class PubmedDatabaseHandler:  
    def __init__(self):
        self.fields = ['pmid', 'year', 'title', 'abstract']
        self.conn = pg_driver.connect(dbname='pubmed', user='biolabs', password='pubtrends', host='localhost')
        self.cursor = self.conn.cursor()
        
    def size(self):
        self.cursor.execute('SELECT COUNT(1) FROM Publications;')
        result = []
        
        for row in self.cursor:
            result.append(row)
            
        return result[0]
    
    def create_citation_count_table(self):
        query = f'SELECT pmid_cited AS pmid, COUNT(1) INTO CitationsCount FROM Citations GROUP BY pmid_cited'
        with self.conn:
            self.cursor.execute(query)
        
    def get_paper_by_id(self, pmid):
        query = f'SELECT {",".join(self.fields)} FROM Publications WHERE pmid={pmid};'
        result = []
        
        self.cursor.execute(query)
        for row in self.cursor:
            result.append(row)
            
        # No duplicate PMIDs should exist, otherwise there is a problem with data storage
        assert len(result) == 1
        
        return result[0]
    
    def get_top_cited_papers(self, year_min=None, year_max=None, limit=10):
        # TODO: use year range
        query = f'SELECT pmid, count FROM CitationsCount ORDER BY count DESC LIMIT {limit}';
        result = []
        
        self.cursor.execute(query)
        for row in self.cursor:
            result.append(row)
            
        assert len(result) == limit
        
        return result
    
    def get_papers_by_keywords(self, *keywords):
        # Drop previous subset
        with self.conn:
            self.cursor.execute('DROP TABLE IF EXISTS PublicationsSubset;')
            
        # Make subset for new keywords
        tsquery = ' & '.join(keywords)
        query = f"SELECT * INTO PublicationsSubset FROM Publications WHERE ta_search @@ to_tsquery('{tsquery}');"
        print(query)
        with self.conn:
            self.cursor.execute(query)
    
    def close():
        self.conn.close()

In [4]:
db_handler = PubmedDatabaseHandler()

#### Number of publications

In [12]:
%time print(db_handler.size())

(28026506,)
Wall time: 31min 56s


#### Get paper by PMID

In [13]:
%time print(db_handler.get_paper_by_id(20545324))

(20545324, 2010, 'The N-terminal integrity is critical for the stability and biological functions of endostatin.', 'Endostatin is an endogenous angiogenesis inhibitor, and amino acid residues H1, H3, H11, and D76 at its N-terminus coordinate with one zinc ion. Recombinant endostatin suffering from N-terminal truncations during Pichia pastoris expression was widely used in previous studies and generated inconsistent antitumor results. However, little attention was paid to the possible alteration on the stability and activity of endostatin caused by N-terminal truncations. In this study, N-terminally truncated forms of endostatin expressed by P. pastoris are identified as N-1, N-3, and N-4, in which one or two of the four zinc-binding residues are lost. The N-terminal truncation of the first amino acid residue (H) does not result in a significant change in the conformation, zinc-binding capacity, thermodynamic stability, or biological activity, while truncations of the first three amino 

#### Top cited papers

In [6]:
papers = db_handler.get_top_cited_papers()
print('PMID, Number of citations')
for paper in papers:
    print(paper[0], paper[1], sep=', ')

PMID, Number of citations
5432063, 50648
14907713, 49773
942051, 37654
11846609, 27085
18156677, 26109
271968, 21827
2231712, 20059
9254694, 19635
7984417, 14356
1202204, 12854


#### TODO: Full text search in title and abstract

In [53]:
db_handler.get_papers_by_keywords('dna', 'methylation', 'clock')

SELECT * INTO PublicationsSubset FROM Publications WHERE ta_search @@ to_tsquery('dna & methylation & clock')
